In [1]:
import os
from data.source.pg_experiment import get_pg_experiment_dataframe
import polars as pl

from models.SimplifiedLightweightCNN import SimplifiedLightweightCNN
%load_ext autoreload
%autoreload 1
%aimport models.SimplifiedLightweightCNN
from models.SimpleCNN_v2 import train, evaluate
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
from path import RESULT_DIRECTORY
import wandb

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [8]:
df_pron, df_tone = get_pg_experiment_dataframe(".ogg")

# Get the target words with val accuracy above 70%
TARGET_WORDS = ["a0", "a1", "a100", "a2", "a3", "a5", "a8"]
dataframe = df_pron.filter(pl.col("word_id").is_in(TARGET_WORDS))

dataframe = dataframe.with_columns([
    pl.struct("word_id").rank("dense").alias("word_id"),
    pl.col("value").cast(pl.Float32) 
])

# Filters
dataframe = dataframe.filter((pl.col("stage") == 1))

N_WORDS = dataframe.select(pl.col("word_id").n_unique()).item()
print(f"Number of unique words: {N_WORDS}")
print(f"Number of samples: {dataframe.shape[0]}")


Number of unique words: 7
Number of samples: 3527


In [9]:
import polars as pl
import numpy as np
from typing import Tuple

def stratified_split(df: pl.DataFrame, label_col: str, train_frac=0.8, val_frac=0.1, seed=42) -> Tuple[pl.DataFrame, pl.DataFrame, pl.DataFrame]:
    classes = df.select(label_col).unique().to_series()
    train_rows, val_rows, test_rows = [], [], []

    rng = np.random.RandomState(seed)

    for cls in classes:
        class_df = df.filter(pl.col(label_col) == cls)
        n = class_df.height
        indices = rng.permutation(n)

        train_end = int(train_frac * n)
        val_end = int((train_frac + val_frac) * n)

        train_rows.append(class_df[indices[:train_end]])
        val_rows.append(class_df[indices[train_end:val_end]])
        test_rows.append(class_df[indices[val_end:]])

    train_df = pl.concat(train_rows)
    val_df = pl.concat(val_rows)
    test_df = pl.concat(test_rows)

    return train_df, val_df, test_df


In [10]:
from typing import Callable

from polars import DataFrame
from dataset import Cast, TorchDataset
from develop import reload_function, reload_module
import pytorch_dataloader
reload_module(pytorch_dataloader)
from pytorch_dataloader import ReshapeCollate, build_collate_fn, PaddingCollate, DefaultCollate
from functools import partial

from transformation import Channels, RMSEnergy, TorchVadLogMelSpec, TorchVadMFCC, ZeroCrossingRate

reload_function(TorchVadMFCC)

TRAIN_SPLIT = 0.6
VAL_SPLIT = 0.2
TEST_SPLIT = 1 - TRAIN_SPLIT - VAL_SPLIT
train_pl, val_pl, test_pl = stratified_split(dataframe, label_col="value", train_frac=TRAIN_SPLIT, val_frac=VAL_SPLIT)

to_dataset: Callable[[DataFrame], TorchDataset] = lambda dataframe: TorchDataset(
    Cast(dataframe.get_column("rec_path"), Channels("stack","multiply")(
            TorchVadMFCC(delta=0),
        )),
    Cast(dataframe.get_column("rec_path"), Channels("cat","multiply")(
            ZeroCrossingRate(),
            RMSEnergy(),
        )),
    Cast(dataframe.get_column("word_id"), lambda x: torch.tensor(x-1, dtype=torch.long)),
    Cast(dataframe.get_column("value"), lambda x: torch.tensor(x).float()),
)

collate_fn = build_collate_fn(
    PaddingCollate(mode="SET_MAX_LEN", max_len=80, pad_dim=2),
    PaddingCollate(mode="SET_MAX_LEN", max_len=160, pad_dim=1),
    DefaultCollate(),
    DefaultCollate(),
)
dataset_train = to_dataset(train_pl)
dataset_val = to_dataset(val_pl)
dataset_test = to_dataset(test_pl)

In [ ]:
from pytorch_dataloader import MemoryLoadedDataLoader
from os import name
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#note, if you are using Windows you MUST set `num_workers=0` - TL;DT multithreading DON'T work in notebooks because Windows DON'T have `fork()`
num_workers = 0 if name == "nt" else 4
train_loader = DataLoader(dataset_train, batch_size=16, shuffle=True, collate_fn=collate_fn, num_workers=num_workers)
val_loader = DataLoader(dataset_val, batch_size=16, shuffle=False, collate_fn=collate_fn, num_workers=num_workers)
test_loader = DataLoader(dataset_test, batch_size=16, shuffle=False, collate_fn=collate_fn, num_workers=num_workers)

for x in next(iter(train_loader)):
    print(x.shape)

cuda
/home/kamil2002/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/505/a2.ogg has no speech segments, using full waveform
/home/kamil2002/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/664/a8.ogg has no speech segments, using full waveform
/home/kamil2002/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/587/a3.ogg has no speech segments, using full waveform
/home/kamil2002/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/345/a2.ogg has no speech segments, using full waveform
/home/kamil2002/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/1059/a2.ogg has no speech segments, using full waveform
torch.Size([16, 1, 40, 80])
torch.Size([16, 2, 160])
torch.Size([16])
torch.Size([16])


In [ ]:
train_loader = MemoryLoadedDataLoader(train_loader, device=device)
print("Loaded train loader into memory")
val_loader = MemoryLoadedDataLoader(val_loader, device=device)
print("Loaded validation loader into memory")

/home/kamil2002/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/493/a1.ogg has no speech segments, using full waveform
/home/kamil2002/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/761/a8.ogg has no speech segments, using full waveform
/home/kamil2002/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/944/a8.ogg has no speech segments, using full waveform
/home/kamil2002/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/587/a1.ogg has no speech segments, using full waveform
/home/kamil2002/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/668/a0.ogg has no speech segments, using full waveform
/home/kamil2002/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/944/a3.ogg has no speech segments, using full waveform/home/kamil2002/Mandarin_Pronunciation_Recognition_Project/data/source/

In [16]:
from models.FusionCNN import ContextFusionCNN
reload_function(ContextFusionCNN)
model = ContextFusionCNN(1,2, num_words=N_WORDS)

In [14]:
wandb.login(key="df5cbdf4ec56162d09c57d7b456e83e24dbd24e1")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/kamil2002/.netrc
wandb: Currently logged in as: fischbach-kamil (fischbach-kamil-pg) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [17]:
# Model variables definition.
pth = "ContextFusionCNN.pth"
lr = 1e-4  # Reduce from 1e-3
epochs = 140
model = model.to(device)
reload_function(train)
reload_function(evaluate)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)  # Add L2 regularization
# Add learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5
)
criterion = nn.BCELoss()

# Start a new wandb run to track this script.
run = wandb.init(
    # name of the run
    name="CFCNN(2) only words with val acc above 70%",
    config={
        "Name": 'ContextFusionCNN',
        "learning_rate": lr,
        "optimizer": "Adam",
        "criterion": "BCELoss",
        "architecture": "ContextFusionCNN",
        "architecture_details": str(model),
        "dataset": "Stage-I-minimized",
        "train_val_test(%)": f'{TRAIN_SPLIT}-{VAL_SPLIT}-{TEST_SPLIT}',
        "epochs": epochs,
    },
)

# Training loop
for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)
    # Update learning rate
    scheduler.step(val_loss)
    # Logging the metadata for each epoch so that the charts can be generated on the dashboard
    run.log({"train_acc": train_acc, "train_loss": train_loss, "val_acc": val_acc, "val_loss": val_loss, })
    print(
        f"Epoch {epoch + 1}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

run.log({"model_eval": model.eval()})
# Saving the model to pth and adding it to the artifacts of the run, there is 5GB of memory on wandb, so we should be fine.
torch.save(model.state_dict(), os.path.join(RESULT_DIRECTORY, pth))
artifact = wandb.Artifact("SimplifiedLightweightCNN-model", type="model")
artifact.add_file(os.path.join(RESULT_DIRECTORY, pth))
run.log_artifact(artifact)

# Finish the run so it gets sent to the remote. You can discover the run right after that on the dashboard.
run.finish()


Epoch 1, Train Loss: 0.6827, Train Acc: 0.5650, Val Loss: 0.6767, Val Acc: 0.5623
Epoch 2, Train Loss: 0.6793, Train Acc: 0.5726, Val Loss: 0.6705, Val Acc: 0.5680
Epoch 3, Train Loss: 0.6761, Train Acc: 0.5702, Val Loss: 0.6656, Val Acc: 0.5708
Epoch 4, Train Loss: 0.6689, Train Acc: 0.5712, Val Loss: 0.6564, Val Acc: 0.5850
Epoch 5, Train Loss: 0.6633, Train Acc: 0.5825, Val Loss: 0.6492, Val Acc: 0.5878
Epoch 6, Train Loss: 0.6617, Train Acc: 0.5868, Val Loss: 0.6420, Val Acc: 0.6048
Epoch 7, Train Loss: 0.6564, Train Acc: 0.6151, Val Loss: 0.6323, Val Acc: 0.6261
Epoch 8, Train Loss: 0.6476, Train Acc: 0.6227, Val Loss: 0.6245, Val Acc: 0.6416
Epoch 9, Train Loss: 0.6460, Train Acc: 0.6047, Val Loss: 0.6201, Val Acc: 0.6785
Epoch 10, Train Loss: 0.6395, Train Acc: 0.6241, Val Loss: 0.6123, Val Acc: 0.7139
Epoch 11, Train Loss: 0.6370, Train Acc: 0.6350, Val Loss: 0.6082, Val Acc: 0.7096
Epoch 12, Train Loss: 0.6278, Train Acc: 0.6525, Val Loss: 0.6015, Val Acc: 0.7040
Epoch 13, Tra

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


train_acc,▁▄▄▄▅▅▅▅▅▅▅▅▆▆▅▆▆▆▆▆▆▆▇▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇██
train_loss,█▇▇▇▇▅▅▅▅▄▄▄▄▄▄▃▃▃▃▄▃▃▃▃▃▃▃▃▃▂▃▂▂▂▂▂▁▂▁▁
val_acc,▁▂▂▄▅▆▆▇▇██████▇██▇▇██▇█▇▇████▇▇▇▇▇██▇█▇
val_loss,█▇▇▆▆▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
model_eval,ContextFusionCNN( ...
train_acc,0.76927
train_loss,0.50721
val_acc,0.74929
val_loss,0.50336


In [18]:
test_loader = MemoryLoadedDataLoader(test_loader, device=device)
print("Loaded test loader into memory")

/home/kamil2002/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/493/a8.ogg has no speech segments, using full waveform
/home/kamil2002/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/550/a100.ogg has no speech segments, using full waveform
/home/kamil2002/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/620/a5.ogg has no speech segments, using full waveform
/home/kamil2002/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/489/a5.ogg has no speech segments, using full waveform
/home/kamil2002/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/772/a1.ogg has no speech segments, using full waveform
/home/kamil2002/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/668/a100.ogg has no speech segments, using full waveform
/home/kamil2002/Mandarin_Pronunciation_Recognition_Project/data/so

In [19]:
model.eval()
test_loss, test_acc = evaluate(model, test_loader, criterion, device)

print("-" * 30)
print(f"FINAL TEST RESULTS")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print("-" * 30)

------------------------------
FINAL TEST RESULTS
Test Loss: 0.5561
Test Accuracy: 0.7195
------------------------------
